# About
- **By**: Tsombou Christian
- **@** : tsombouchris@gmail.com
- **linkedIn**: https://www.linkedin.com/in/tsombouchris/

This Notebook performs an Scrapping of livechennei website in order to obtain data about load shedding schedule in chennai from 2012 to 2022

- **Data Source**: [livechennai website](https://www.livechennai.com/powercut_schedule.asp)
- **Reference**: We Used the [Selenium library](https://selenium-python.readthedocs.io/) and [Beautifull Soup library](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) for scraping the website


_Process:_

At the time of the scrapping, there were 140 pages of schedule with almost 10 schedule per page and for each there was one correspondin page indicating the various localities concerned by the load schedding.
the process is describe in the commented code cells below

 _Result:_
 - A 2463 entries and 11 Columns dataset store in a .csv file

 _Challenges:_

 We had include dirty data in the scrapping process in order to capture the locations wich where not clearly formatted and thus identifiable, this obliged us to proceed to a long data clening process after scrapping.


NB: This Notebook was produced within the context of The Omdena [Chennai Chapter project on Electricity Power Outage Analysis](https://github.com/OmdenaAI/chennai-india-power-outage)

# Web Scrapping Power outages in Chennai India

### Reading the web page into python

In [7]:
# ##   Installing useful Libraries

# ! pip install requests
# ! pip install -U selenium

In [1]:
import requests
r = requests.get('https://www.livechennai.com/powercut_schedule.asp')

c:\Users\user\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
c:\Users\user\Anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


In [2]:
# print the first 500 characters of the HTML
print(r.text[0:500])


	<html>
	<head>
	<meta name="ROBOTS" content="follow,index,all">
	<meta http-equiv="X-UA-Compatible" content="IE=8">
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<META HTTP-EQUIV="Content-Language" CONTENT="en">
	<META NAME="Description" CONTENT="www.livechennai.com - Power Shut Down Details of Chennai, Tamilnadu, power shutdown in chennai, today power shutdown in chennai">
	<META NAME="Keywords" CONTENT="  Power Cut Schedules in Chennai, Chennai Power Cut Sche


## Parsing the HTML using Beautiful Soup and Selenium

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import time

In [4]:
url = 'https://www.livechennai.com/powercut_schedule.asp'

In [5]:
# setting the webdriver with Chrome
driver = webdriver.Chrome()

In [30]:
# Access the webpage
driver.get(url)



## Collecting all of the records

In [7]:
#   Avalaible page numbers on the current page
results_page = soup.find_all('a', attrs={'class':'classic_pageLink_deselect'})
# Catching the last page number
last_page_number = int(results_page[2].text)


In [31]:
pm_am =['pm', 'am', 'A.M', 'P.M','P.M.', 'A.M.', 'PM', 'AM', 'p.m.', 'a.m.', 'p.m,', 'a.m,', 'P.M,', 'A.M,']

In [9]:
# creating an empty dataframe to store results
df_ = pd.DataFrame(columns=['PO_day', 'PO_date', 'PO_start_time', 'PO_end_time', 'PO_posting_date', 'PO_posting_time', 'PO_locations_link'])
df_

,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link


In [32]:
i=0

while i <= last_page_number:
    print(i)

    content = driver.find_elements(By.CLASS_NAME, 'new_page')
    title_content = driver.find_elements(By.CLASS_NAME, 'w')
    

    for k in range(len(content)):
        
        # Power outage message title
        PO_message_title = title_content[k].find_element(By.TAG_NAME,'p').find_element(By.TAG_NAME,'a').text
        # Spliting the Power Outage message title to gather detailed information
        PO_message_title_list = PO_message_title.split()

        # Power Outage message
        PO_message = content[k].find_element(By.CLASS_NAME, 'cont').text
        # Spliting the Power Outage message to gather detailed information
        PO_message_list = PO_message.split()

        ### Power outage day of the week
        PO_day = PO_message_title_list[-2]

        ### Power outage date 
        PO_date = PO_message_title_list[-1].strip('()').replace('.','/').replace('-','/')

        ### Power outage starting time 
        try:
            index_pm_am = [PO_message_list[i].lower() in pm_am  for i in range(len(PO_message_list))].index(True)
        except ValueError:
            continue

        
        PO_start_time = PO_message_list[index_pm_am -1 ] + " " + PO_message_list[index_pm_am]
        
        ### Power outage ending time
        try:
            PO_end_time = PO_message_list[index_pm_am + 2] + " " + PO_message_list[index_pm_am + 3]
        except IndexError:
            PO_end_time = PO_message_list[index_pm_am + 2]

        ### Power outage reason
        #PO_reason = " ".join(PO_message_list[PO_message_list.index('for')+1 :])

        # Power Outage posting data and time message
        power_outage_message_posted_on = content[k].find_element(By.CLASS_NAME, 'read').text
        power_outage_message_posted_on_list = power_outage_message_posted_on.split()

        ### Power outage posting date and time
        posting_date = power_outage_message_posted_on_list[3]
        posting_time = " ".join(power_outage_message_posted_on_list[4:])

        ## locations page link
        ###### we first get the link of the location page with schedule title
        location_page_link = title_content[k].find_element(By.TAG_NAME,'p').find_element(By.TAG_NAME,'a').get_attribute('href')


        # A list will all information gathered and then a dictionnary column_name:value to append to the result dataframe
        row=[PO_day, PO_date, PO_start_time, PO_end_time, posting_date, posting_time, location_page_link]
        row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
        df_ = df_.append(row_dict, ignore_index=True)
        print(row)
        
    
    if i == 140:
    #if i == last_page_number:
        break

    # Catching the next page reference
    next_page_button = driver.find_element(By.CLASS_NAME, 'classic_pageLink_last')
    #next page link
    next_page_link = next_page_button.get_attribute('href')
    driver.get(next_page_link)
    time.sleep(4)
    i +=1
    print(next_page_link,i)


130
['on', '10/05/2014', '9.00 AM', '2.00 PM', '09/May/2014', '4:13:21 PM', 'https://www.livechennai.com/detailnews.asp?newsid=11160&catid=10']
['on', '09/05/2014', '9.00 AM', '2.00 PM', '08/May/2014', '3:25:55 PM', 'https://www.livechennai.com/detailnews.asp?newsid=11125&catid=10']
['on', '08/05/2014', '9.00 AM', '2.00 PM', '07/May/2014', '12:09:01 PM', 'https://www.livechennai.com/detailnews.asp?newsid=11093&catid=10']
['on', '07/05/2014', '9.00 AM', '2.00 PM', '06/May/2014', '11:18:47 AM', 'https://www.livechennai.com/detailnews.asp?newsid=11066&catid=10']
['on', '03/05/2014', '9.00 A.M.', '2.00 P.M.', '02/May/2014', '3:51:23 PM', 'https://www.livechennai.com/detailnews.asp?newsid=11002&catid=10']
['Today', '30/4/2014', '9.00 A.M.', '2.00 P.M.', '30/Apr/2014', '11:17:24 AM', 'https://www.livechennai.com/detailnews.asp?newsid=10939&catid=10']
['Today', '29/04/2014', '9.00 A.M.', '2.00 P.M.', '29/Apr/2014', '12:08:51 PM', 'https://www.livechennai.com/detailnews.asp?newsid=10928&catid=

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".classic_pageLink_last"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E56463+2188387]
	Ordinal0 [0x00DEE461+1762401]
	Ordinal0 [0x00D03D78+802168]
	Ordinal0 [0x00D31880+989312]
	Ordinal0 [0x00D31B1B+989979]
	Ordinal0 [0x00D5E912+1173778]
	Ordinal0 [0x00D4C824+1099812]
	Ordinal0 [0x00D5CC22+1166370]
	Ordinal0 [0x00D4C5F6+1099254]
	Ordinal0 [0x00D26BE0+945120]
	Ordinal0 [0x00D27AD6+948950]
	GetHandleVerifier [0x010F71F2+2712546]
	GetHandleVerifier [0x010E886D+2652765]
	GetHandleVerifier [0x00EE002A+520730]
	GetHandleVerifier [0x00EDEE06+516086]
	Ordinal0 [0x00DF468B+1787531]
	Ordinal0 [0x00DF8E88+1805960]
	Ordinal0 [0x00DF8F75+1806197]
	Ordinal0 [0x00E01DF1+1842673]
	BaseThreadInitThunk [0x7608FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A37A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77A37A6E+238]


In [33]:
df_

,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link
0,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...
1,Saturday,16/07/2022,9 am,"2 pm,",16/Jul/2022,8:47:41 AM,https://www.livechennai.com/detailnews.asp?new...
2,Friday,15/07/2022,09.00 am,02.00 pm,14/Jul/2022,6:30:48 PM,https://www.livechennai.com/detailnews.asp?new...
3,Thursday,14/07/2022,09.00 am,02.00 pm,14/Jul/2022,9:46:46 AM,https://www.livechennai.com/detailnews.asp?new...
4,Wednesday,13/07/2022,09.00 am,02.00 pm,12/Jul/2022,5:06:56 PM,https://www.livechennai.com/detailnews.asp?new...
...,...,...,...,...,...,...,...
550,On,24/12/2013,9.00 A.M.,5.00 P.M.,23/Dec/2013,2:21:47 PM,https://www.livechennai.com/detailnews.asp?new...
551,On,21/12/2013,9.00 A.M.,5.00 P.M.,19/Dec/2013,4:01:20 PM,https://www.livechennai.com/detailnews.asp?new...
552,On,20/12/2013,9.00 A.M.,5.00 P.M.,18/Dec/2013,6:24:20 PM,https://www.livechennai.com/detailnews.asp?new...
553,On,19/12/2013,9.00 A.M.,5.00 P.M.,17/Dec/2013,3:41:11 PM,https://www.livechennai.com/detailnews.asp?new...


In [52]:
# remove duplicates
df_[df_.duplicated()]

,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link
490,on,10/05/2014,9.00 AM,2.00 PM,09/May/2014,4:13:21 PM,https://www.livechennai.com/detailnews.asp?new...
491,on,09/05/2014,9.00 AM,2.00 PM,08/May/2014,3:25:55 PM,https://www.livechennai.com/detailnews.asp?new...
492,on,08/05/2014,9.00 AM,2.00 PM,07/May/2014,12:09:01 PM,https://www.livechennai.com/detailnews.asp?new...
493,on,07/05/2014,9.00 AM,2.00 PM,06/May/2014,11:18:47 AM,https://www.livechennai.com/detailnews.asp?new...
494,on,10/05/2014,9.00 AM,2.00 PM,09/May/2014,4:13:21 PM,https://www.livechennai.com/detailnews.asp?new...
495,on,09/05/2014,9.00 AM,2.00 PM,08/May/2014,3:25:55 PM,https://www.livechennai.com/detailnews.asp?new...
496,on,08/05/2014,9.00 AM,2.00 PM,07/May/2014,12:09:01 PM,https://www.livechennai.com/detailnews.asp?new...
497,on,07/05/2014,9.00 AM,2.00 PM,06/May/2014,11:18:47 AM,https://www.livechennai.com/detailnews.asp?new...


In [56]:
df_.drop_duplicates(inplace=True)
df_.shape

(547, 7)

### Exporting the dataset to a CSV file

In [57]:
df_.to_csv('v3_outages_in_chennai.csv', index=False, encoding='utf-8')

In [42]:
df1_ = pd.DataFrame(columns=['PO_locations_link1', 'location'])
df1_

,PO_locations_link1,location


In [43]:
#Extracting the locations
for link in df_['PO_locations_link']:
    driver.get(link)
    time.sleep(6)
    ###### we extract all content in bold among which the locations (the cleaning will be done after)
    locations = driver.find_element(By.CLASS_NAME,'dark').find_elements(By.TAG_NAME, 'b')
    locations_text = [locations[i].text for i in range(len(locations))]

    # A list will all information gathered and then a dictionnary column_name:value
        #  to append to the result dataframe for each location in location_text
    for location_text in locations_text:
        row=[link, location_text]
        row_dict = {df1_.columns[i]:row[i] for i in range(len(row))}
        df1_ = df1_.append(row_dict, ignore_index=True)

    print(df1_.shape)



(4, 2)
(12, 2)
(27, 2)
(41, 2)
(54, 2)
(67, 2)
(82, 2)
(94, 2)
(107, 2)
(117, 2)
(127, 2)
(135, 2)
(153, 2)
(167, 2)
(181, 2)
(192, 2)
(207, 2)
(223, 2)
(230, 2)
(240, 2)
(251, 2)
(265, 2)
(271, 2)
(281, 2)
(288, 2)
(294, 2)
(303, 2)
(311, 2)
(321, 2)
(329, 2)
(333, 2)
(342, 2)
(351, 2)
(355, 2)
(364, 2)
(366, 2)
(369, 2)
(375, 2)
(380, 2)
(386, 2)
(388, 2)
(395, 2)
(401, 2)
(409, 2)
(413, 2)
(420, 2)
(423, 2)
(428, 2)
(433, 2)
(440, 2)
(448, 2)
(457, 2)
(468, 2)
(470, 2)
(482, 2)
(494, 2)
(506, 2)
(513, 2)
(519, 2)
(524, 2)
(532, 2)
(535, 2)
(538, 2)
(544, 2)
(548, 2)
(558, 2)
(566, 2)
(574, 2)
(579, 2)
(584, 2)
(587, 2)
(590, 2)
(592, 2)
(603, 2)
(606, 2)
(610, 2)
(616, 2)
(619, 2)
(623, 2)
(627, 2)
(631, 2)
(638, 2)
(639, 2)
(641, 2)
(650, 2)
(656, 2)
(659, 2)
(663, 2)
(669, 2)
(672, 2)
(677, 2)
(681, 2)
(687, 2)
(692, 2)
(695, 2)
(698, 2)
(702, 2)
(705, 2)
(713, 2)
(719, 2)
(720, 2)
(722, 2)
(724, 2)
(726, 2)
(728, 2)
(731, 2)
(733, 2)
(735, 2)
(739, 2)
(741, 2)
(743, 2)
(745, 2)
(

In [49]:
df1_

,PO_locations_link1,location
0,https://www.livechennai.com/detailnews.asp?new...,Tuesday (19.07.2022)
1,https://www.livechennai.com/detailnews.asp?new...,TAMBARAM AREA:
2,https://www.livechennai.com/detailnews.asp?new...,T. NAGAR AREA:
3,https://www.livechennai.com/detailnews.asp?new...,ADYAR AREA:
4,https://www.livechennai.com/detailnews.asp?new...,"Egmore, Tambaram, Porur, Adyar, Ponneri , Avad..."
...,...,...
2716,https://www.livechennai.com/detailnews.asp?new...,NANGANALLUR AREA
2717,https://www.livechennai.com/detailnews.asp?new...,NANDANAM AREA
2718,https://www.livechennai.com/detailnews.asp?new...,ROYAPURAM AREA
2719,https://www.livechennai.com/detailnews.asp?new...,K.K. NAGAR AREA


In [60]:
# drop duplicates
df1_.duplicated().sum()
df1_.drop_duplicates(inplace=True)
df1_.shape

(2674, 2)

In [61]:
# Export to csv
df1_.to_csv('v3_locations_outages_in_chennai.csv', index=False, encoding='utf-8')

In [62]:
# merge the two dataframe on location link
df_merged = pd.merge(left=df_, right=df1_, left_on='PO_locations_link', right_on='PO_locations_link1')

In [63]:
df_merged.shape

(2674, 9)

In [68]:
df_merged.head()

,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link,PO_locations_link1,location,location_split
0,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,Tuesday (19.07.2022),"[Tuesday, (19.07.2022)]"
1,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,TAMBARAM AREA:,"[TAMBARAM, AREA:]"
2,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,T. NAGAR AREA:,"[T., NAGAR, AREA:]"
3,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,ADYAR AREA:,"[ADYAR, AREA:]"
4,Saturday,16/07/2022,9 am,"2 pm,",16/Jul/2022,8:47:41 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,"Egmore, Tambaram, Porur, Adyar, Ponneri , Avad...","[Egmore,, Tambaram,, Porur,, Adyar,, Ponneri, ..."


In [263]:
df_merged = df_merged.drop(columns=['PO_locations_link1'])

In [157]:
# split the location text in a new column
df_merged['location_split'] = df_merged.location.apply(lambda x: x.strip().split())

,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link,PO_locations_link1,location,location_split
292,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,Power supply will be suspended in the followin...,"[Power, supply, will, be, suspended, in, the, ..."
293,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,AMBATTUR AREA:,"[AMBATTUR, AREA:]"
294,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,PORUR AREA:,"[PORUR, AREA:]"
295,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,KILPAUK AREA:,"[KILPAUK, AREA:]"
296,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,SEMBIUM AREA :,"[SEMBIUM, AREA, :]"
297,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,TAMBARAM AREA :,"[TAMBARAM, AREA, :]"
298,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,GUINDY:,[GUINDY:]
299,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,KK NAGAR:,"[KK, NAGAR:]"
300,Thursday,16/06/2022,09.00 am,02.00 pm,15/Jun/2022,9:51:09 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,ANNA NAGAR/MADURAIVOYAL AREA:,"[ANNA, NAGAR/MADURAIVOYAL, AREA:]"


In [ ]:
# check for empty locations
is_empty_loc = df_merged.location == ''
is_empty_loc.sum()

In [ ]:
# delete empty locations row
df_merged = df_merged[~is_empty_loc]
df_merged.info()

In [137]:
bad_locations_set = {'friday', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday', 'Friday', 'Monday',
 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday','Power', 'supply', 'Supply'}

In [199]:
# identify some dirty data for location with bad_locations_set
dirty_loc_Ps = df_merged.location_split.apply(lambda x: True if bad_locations_set.intersection(set(x)) else False)
dirty_loc_Ps.sum()
 

292    MYLAPORE
Name: location, dtype: object
582    KK Nagar / Ramasamy Road
Name: location, dtype: object


In [ ]:
# treat two particular cases
for loca in df_merged.location_split[dirty_loc_Ps]:
    if 'MYLAPORE' in loca:
       index1 = df_merged[df_merged.location_split.apply(lambda x: x == loca)].index
       df_merged.loc[index1,'location'] = 'MYLAPORE'
       print(df_merged.loc[index1,'location'])

    if 'Ramasamy' in loca:
        index2 = df_merged[df_merged.location_split.apply(lambda x: x == loca)].index
        df_merged.loc[index2,'location'] = 'KK Nagar / Ramasamy Road'
        print(df_merged.loc[index2,'location'])

In [202]:
# update the location_split column
df_merged['location_split'] = df_merged.location.apply(lambda x: x.strip().split())

In [203]:
# reidentify some dirty data for location with bad_locations_set after treating the two particular cases
dirty_loc_Ps = df_merged.location_split.apply(lambda x: True if bad_locations_set.intersection(set(x)) else False)
dirty_loc_Ps.sum()

120

In [204]:
#drop all rows with the condition :'dirty_loc_Ps'
df_merged = df_merged[~dirty_loc_Ps]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2471 entries, 1 to 2673
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PO_day              2471 non-null   object
 1   PO_date             2471 non-null   object
 2   PO_start_time       2471 non-null   object
 3   PO_end_time         2471 non-null   object
 4   PO_posting_date     2471 non-null   object
 5   PO_posting_time     2471 non-null   object
 6   PO_locations_link   2471 non-null   object
 7   PO_locations_link1  2471 non-null   object
 8   location            2471 non-null   object
 9   location_split      2471 non-null   object
dtypes: object(10)
memory usage: 212.4+ KB


In [236]:
# dirty locations with len of list after splitting GT 7
dirty_loc_len7 = df_merged.location_split.apply(lambda x: len(x)>7)

# treat three particular cases
for loca in df_merged.location_split[dirty_loc_len7]:
    if 'hills/Attanthangal,' in loca:
       index1 = df_merged[df_merged.location_split.apply(lambda x: x == loca)].index
       df_merged.loc[index1,'location'] = 'Mylapore'
       print(df_merged.loc[index1,'location'])
       

    if 'Mathur:' in loca:
        index2 = df_merged[df_merged.location_split.apply(lambda x: x == loca)].index
        df_merged.loc[index2,'location'] = 'Mathur'
        print(df_merged.loc[index2,'location'])
        print(loca)

    if 'Adyar' in loca:
        index3 = df_merged[df_merged.location_split.apply(lambda x: x == loca)].index
        df_merged.loc[index3,'location'] = 'Adyar'
        print(df_merged.loc[index3,'location'])
        
       
# update the location_split column
df_merged['location_split'] = df_merged.location.apply(lambda x: x.strip().split())


166    Mylapore
Name: location, dtype: object
1918    Mathur
Name: location, dtype: object
['Mathur:', '1,', '2,', '3rd', 'main', 'road', 'MMDA,', 'MMDA', 'Mathur.']
1919    Adyar
Name: location, dtype: object


In [237]:
# reidentify some dirty locations with len of list after splitting GT 7
dirty_loc_len7 = df_merged.location_split.apply(lambda x: len(x)>7)
dirty_loc_len7.sum()

8

In [238]:
#drop all rows with the condition :'dirty_loc_len7'
df_merged = df_merged[~dirty_loc_len7]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2463 entries, 1 to 2673
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PO_day              2463 non-null   object
 1   PO_date             2463 non-null   object
 2   PO_start_time       2463 non-null   object
 3   PO_end_time         2463 non-null   object
 4   PO_posting_date     2463 non-null   object
 5   PO_posting_time     2463 non-null   object
 6   PO_locations_link   2463 non-null   object
 7   PO_locations_link1  2463 non-null   object
 8   location            2463 non-null   object
 9   location_split      2463 non-null   object
dtypes: object(10)
memory usage: 211.7+ KB


In [239]:
for l in df_merged[df_merged.location_split.apply(lambda x: len(x)>6)].location_split:
    print(len(l),':', l)
    time.sleep(2)

7 : ['SEMBIUM', '/', 'RETTERI', '/', 'PERAMBUR', '/', 'CMBT:']
7 : ['Avadi/Pattabiram,', 'Sothuperumbedu,', 'Ambattur,', 'Sembium,', 'Villivakkam,', 'Vyasarpadi,', 'Guindy/Nanganallur']
7 : ['TIDAL', 'PARK', '/SRP', 'TOOLS', 'OMR', 'AREA', ':']


In [254]:
# strip the location text 
df_merged['location'] = df_merged.location.apply(lambda x: x.strip(' ,:.'))
df_merged

c:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link,PO_locations_link1,location,location_split,location_split_coma
1,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,TAMBARAM AREA,"[TAMBARAM, AREA:]",[TAMBARAM AREA]
2,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,T. NAGAR AREA,"[T., NAGAR, AREA:]",[T. NAGAR AREA]
3,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,ADYAR AREA,"[ADYAR, AREA:]",[ADYAR AREA]
5,Saturday,16/07/2022,9 am,"2 pm,",16/Jul/2022,8:47:41 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,Egmore,[Egmore],[Egmore]
6,Saturday,16/07/2022,9 am,"2 pm,",16/Jul/2022,8:47:41 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,Tambaram,[Tambaram],[Tambaram]
...,...,...,...,...,...,...,...,...,...,...,...
2669,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,NANGANALLUR AREA,"[NANGANALLUR, AREA]",[NANGANALLUR AREA]
2670,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,NANDANAM AREA,"[NANDANAM, AREA]",[NANDANAM AREA]
2671,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,ROYAPURAM AREA,"[ROYAPURAM, AREA]",[ROYAPURAM AREA]
2672,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,K.K. NAGAR AREA,"[K.K., NAGAR, AREA]",[K.K. NAGAR AREA]


In [258]:
# split the location text in a new column
df_merged['location_split_coma'] = df_merged.location.apply(lambda x: x.replace(' and ',',').split(','))
df_merged

c:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PO_day,PO_date,PO_start_time,PO_end_time,PO_posting_date,PO_posting_time,PO_locations_link,PO_locations_link1,location,location_split,location_split_coma
1,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,TAMBARAM AREA,"[TAMBARAM, AREA:]",[TAMBARAM AREA]
2,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,T. NAGAR AREA,"[T., NAGAR, AREA:]",[T. NAGAR AREA]
3,Tuesday,19/07/2022,09.00 am,02.00 pm,18/Jul/2022,9:26:34 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,ADYAR AREA,"[ADYAR, AREA:]",[ADYAR AREA]
5,Saturday,16/07/2022,9 am,"2 pm,",16/Jul/2022,8:47:41 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,Egmore,[Egmore],[Egmore]
6,Saturday,16/07/2022,9 am,"2 pm,",16/Jul/2022,8:47:41 AM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,Tambaram,[Tambaram],[Tambaram]
...,...,...,...,...,...,...,...,...,...,...,...
2669,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,NANGANALLUR AREA,"[NANGANALLUR, AREA]",[NANGANALLUR AREA]
2670,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,NANDANAM AREA,"[NANDANAM, AREA]",[NANDANAM AREA]
2671,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,ROYAPURAM AREA,"[ROYAPURAM, AREA]",[ROYAPURAM AREA]
2672,on,18/12/2013,9.00 A.M.,5.00 P.M.,16/Dec/2013,4:07:22 PM,https://www.livechennai.com/detailnews.asp?new...,https://www.livechennai.com/detailnews.asp?new...,K.K. NAGAR AREA,"[K.K., NAGAR, AREA]",[K.K. NAGAR AREA]


In [277]:
# create an empty dataframe to take in consideration locations that are multiple
df2_ = pd.DataFrame(columns=['PO_day', 'PO_date', 'PO_start_time', 'PO_end_time', 'PO_posting_date', 'PO_posting_time',
 'PO_locations_link', 'location'])

In [282]:
cols = ['PO_day', 'PO_date', 'PO_start_time', 'PO_end_time', 'PO_posting_date', 'PO_posting_time',
 'PO_locations_link']
location_split_comaGT1 = df_merged.location_split_coma.apply(lambda x: len(x)>1)
df_merged[location_split_comaGT1]

for locations in df_merged.location_split_coma[location_split_comaGT1]:
       index_ = df_merged[df_merged.location_split_coma.apply(lambda x: x == locations)].index
       for location in locations:
              row = df_merged.loc[index_,cols]
              row['location'] = location
              #row.append(location)
              #row_dict = {df2_.columns[i]:row[i] for i in range(len(row))}
              #df2_ = df2_.append(row_dict, ignore_index=True)
              df2_ = pd.concat([df2_,row],ignore_index=True)
              
             
       

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(12, 8)
(13, 8)
(14, 8)
(15, 8)
(16, 8)
(17, 8)
(18, 8)
(19, 8)
(20, 8)
(21, 8)
(22, 8)
(23, 8)
(24, 8)
(25, 8)
(26, 8)
(27, 8)
(28, 8)
(29, 8)
(30, 8)
(31, 8)
(32, 8)
(33, 8)
(34, 8)
(35, 8)
(36, 8)
(37, 8)
(38, 8)
(39, 8)
(40, 8)
(41, 8)
(42, 8)
(43, 8)
(44, 8)
(45, 8)
(46, 8)
(47, 8)
(48, 8)
(49, 8)
(50, 8)
(51, 8)
(52, 8)
(53, 8)
(54, 8)
(55, 8)
(56, 8)
(57, 8)
(58, 8)
(59, 8)
(60, 8)
(61, 8)


In [297]:
# drop unecessary columns location_split and location_split_coma
df_merged = df_merged.drop(columns=['location_split', 'location_split_coma'])

In [305]:
# concatenate df2_ and df_merged

df_merged2 = pd.concat([df_merged, df2_], ignore_index=True)

In [309]:
# replace area wording by ''
to_replace_list = {'AREA', 'AREAS', 'Area', 'Areas', 'area', 'areas', 'AREA:', 'AREAS:', 'Area:', 'Areas:', 'area:', 'areas:'}
for rep in to_replace_list:
    df_merged2['location'] = df_merged2['location'].apply(lambda x: x.replace(rep,''))

In [311]:
 df_merged2.to_csv('17.07.22_power_outages_chennai_located.csv', index=False, encoding='utf-8')